<a href="https://colab.research.google.com/github/yukinaga/minnano_ai/blob/master/section_7/ml_libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 機械学習ライブラリ
機械学習ライブラリ、KerasとPyTorchのコードを紹介します。  
今回はコードの詳しい解説は行いませんが、実装の大まかな流れを把握しましょう。


## ● Kerasのコード
以下のコードは、Kerasによるシンプルなニューラルネットワークの実装です。  
Irisの各花を、SetosaとVersicolorに分類します。  
以下のコードでは、`Sequential`でモデルを作り、層や活性化関数を追加しています。

In [ ]:
import numpy as np

from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
iris_data = iris.data
sl_data = iris_data[:100, 0] # SetosaとVersicolor、Sepal length
sw_data = iris_data[:100, 1] # SetosaとVersicolor、Sepal width

# 平均値を0に
sl_ave = np.average(sl_data)  # 平均値
sl_data -= sl_ave  # 平均値を引く
sw_ave = np.average(sw_data)
sw_data -= sw_ave

# 入力をリストに格納
input_data = []
correct_data = []
for i in range(100):
    input_data.append([sl_data[i], sw_data[i]])
    correct_data.append([iris.target[i]])

# 訓練データとテストデータに分割
input_data = np.array(input_data)  # NumPyの配列に変換
correct_data = np.array(correct_data)
x_train, x_test, t_train, t_test = train_test_split(input_data, correct_data)

# ------ ここからKerasのコード ------
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(2, input_dim=2)) # 入力:2、中間層のニューロン数:2
model.add(Activation("sigmoid")) # シグモイド関数
model.add(Dense(1)) # 出力層のニューロン数:1
model.add(Activation("sigmoid")) # シグモイド関数
model.compile(optimizer=SGD(lr=0.3), loss="mean_squared_error", metrics=["accuracy"])

model.fit(x_train, t_train, epochs=32, batch_size=1)  # 訓練

loss, accuracy = model.evaluate(x_test, t_test)
print("正解率: " + str(accuracy*100) + "%")

## ● PyTorchのコード
以下のコードは、PyTorchよるシンプルなニューラルネットワークの実装です。  
Irisの各花を、SetosaとVersicolorに分類します。  
以下のコードでは、Kerasと同様に`Sequential`でモデルを作り、層や活性化関数を並べています。  
PyTorchでは、入力や正解をTensor形式のデータに変換する必要があります。

In [1]:
import numpy as np

from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
iris_data = iris.data
sl_data = iris_data[:100, 0] # SetosaとVersicolor、Sepal length
sw_data = iris_data[:100, 1] # SetosaとVersicolor、Sepal width

# 平均値を0に
sl_ave = np.average(sl_data)  # 平均値
sl_data -= sl_ave  # 平均値を引く
sw_ave = np.average(sw_data)
sw_data -= sw_ave

# 入力をリストに格納
input_data = []
correct_data = []
for i in range(100):
    input_data.append([sl_data[i], sw_data[i]])
    correct_data.append([iris.target[i]])

# 訓練データとテストデータに分割
input_data = np.array(input_data)  # NumPyの配列に変換
correct_data = np.array(correct_data)
x_train, x_test, t_train, t_test = train_test_split(input_data, correct_data)

# ------ ここからPyTorchのコード ------
import torch
from torch import nn
from torch import optim

# Tensorに変換
x_train = torch.tensor(x_train, dtype=torch.float32)
t_train = torch.tensor(t_train, dtype=torch.float32) 
x_test = torch.tensor(x_test, dtype=torch.float32)
t_test = torch.tensor(t_test, dtype=torch.float32) 

net = nn.Sequential(
    nn.Linear(2, 2), # 入力:2、中間層のニューロン数:2
    nn.Sigmoid(), # シグモイド関数
    nn.Linear(2, 1), # 出力層のニューロン数:1
    nn.Sigmoid() # シグモイド関数
)

loss_fnc = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.3)

# 1000エポック学習
for i in range(1000):

    # 勾配を0に
    optimizer.zero_grad()
    
    # 順伝播
    y_train = net(x_train)
    y_test = net(x_test)
    
    # 誤差を求める
    loss_train = loss_fnc(y_train, t_train)
    loss_test = loss_fnc(y_test, t_test)

    # 逆伝播（勾配を求める）
    loss_train.backward()
    
    # パラメータの更新
    optimizer.step()

    if i%100 == 0:
        print("Epoch:", i, "Loss_Train:", loss_train.item(), "Loss_Test:", loss_test.item())

y_test = net(x_test)
count = ((y_test.detach().numpy()>0.5) == (t_test.detach().numpy()==1.0)).sum().item()
print("正解率: " + str(count/len(y_test)*100) + "%")

Epoch: 0 Loss_Train: 0.2594730257987976 Loss_Test: 0.25115740299224854
Epoch: 100 Loss_Train: 0.22102932631969452 Loss_Test: 0.24556970596313477
Epoch: 200 Loss_Train: 0.15973171591758728 Loss_Test: 0.18285588920116425
Epoch: 300 Loss_Train: 0.09611750394105911 Loss_Test: 0.11673138290643692
Epoch: 400 Loss_Train: 0.059146951884031296 Loss_Test: 0.07832591235637665
Epoch: 500 Loss_Train: 0.04001173377037048 Loss_Test: 0.05797707661986351
Epoch: 600 Loss_Train: 0.029384208843111992 Loss_Test: 0.04636969417333603
Epoch: 700 Loss_Train: 0.022904139012098312 Loss_Test: 0.03919101133942604
Epoch: 800 Loss_Train: 0.01863018423318863 Loss_Test: 0.03447197005152702
Epoch: 900 Loss_Train: 0.015633489936590195 Loss_Test: 0.031232118606567383
正解率: 96.0%
